# numero di nodi variabile per ciascun grafo

In [10]:
import torch
from torch_geometric.data import Data, DataLoader
import random

num_graphs = 50
num_features = 8
batch_size= 25

# Creazione di una lista di oggetti Data
graph_data_list = []
for i in range(num_graphs):
    # Creazione di un grafo con nodi e bordi casuali
    num_nodes = random.randint(10, 20)
    edge_index = torch.randint(num_nodes, (2, num_nodes * 2))
    x = torch.randn(num_nodes, num_features)
    y = torch.randint(2, (num_nodes, 1))
    graph_data_list.append(Data(x=x, edge_index=edge_index, y=y))

# Creazione del DataLoader per il dataset di grafi
loader = DataLoader(graph_data_list, batch_size=batch_size, shuffle=True)


/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/libpyg.so: undefined symbol: _ZN2at4_ops12split_Tensor4callERKNS_6TensorEN3c106SymIntEl
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_scatter_cpu.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_diag_cpu.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slo

OSError: /usr/local/lib/python3.10/dist-packages/torch_cluster/_grid_cpu.so: undefined symbol: _ZN2at4_ops6narrow4callERKNS_6TensorElN3c106SymIntES6_

In [16]:
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Definizione della rete neurale
class MyGraphNet(torch.nn.Module):
    def __init__(self, num_features, hidden_size, num_classes):
        super(MyGraphNet, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_size)
        self.conv2 = GCNConv(hidden_size, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Creazione della rete neurale e del DataLoader
hidden_size = 16
num_classes = 2
net = MyGraphNet(num_features, hidden_size, num_classes)
loader = DataLoader(graph_data_list, batch_size=batch_size, shuffle=True)

# Definizione della funzione di loss e dell'ottimizzatore
criterion = torch.nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

# Iterazione sul dataset di grafi
for epoch in range(10000):
    runloss = 0
    for i, data in enumerate(loader):
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out, data.y.view(-1))
        loss.backward()
        optimizer.step()
        runloss += loss.item()
    if epoch % 100 == 0:
        print('Epoch {}, Iteration {}, Loss {}'.format(epoch, i, runloss/len(loader)))


Epoch 0, Iteration 1, Loss 0.7506682872772217
Epoch 100, Iteration 1, Loss 0.6417058706283569
Epoch 200, Iteration 1, Loss 0.6081125736236572
Epoch 300, Iteration 1, Loss 0.5944410860538483
Epoch 400, Iteration 1, Loss 0.5926471948623657
Epoch 500, Iteration 1, Loss 0.582650750875473
Epoch 600, Iteration 1, Loss 0.576681524515152
Epoch 700, Iteration 1, Loss 0.573774129152298
Epoch 800, Iteration 1, Loss 0.5740209519863129
Epoch 900, Iteration 1, Loss 0.5676935017108917
Epoch 1000, Iteration 1, Loss 0.5607938170433044
Epoch 1100, Iteration 1, Loss 0.5604302883148193
Epoch 1200, Iteration 1, Loss 0.55526202917099
Epoch 1300, Iteration 1, Loss 0.5494658946990967
Epoch 1400, Iteration 1, Loss 0.5475896596908569
Epoch 1500, Iteration 1, Loss 0.5465902090072632
Epoch 1600, Iteration 1, Loss 0.5456690192222595
Epoch 1700, Iteration 1, Loss 0.544447124004364
Epoch 1800, Iteration 1, Loss 0.5435647666454315
Epoch 1900, Iteration 1, Loss 0.547014981508255
Epoch 2000, Iteration 1, Loss 0.5435128